### Import Libraries

In [1]:
!pip install lightgbm
!pip install random
!pip install os
!pip install hyperopt

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
import pandas as pd
import random
import os
import numpy as np
from functools import partial
from lightgbm import LGBMRegressor
from hyperopt import fmin, hp, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU 할당

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [5]:
def dataset_split_X_y(df):    
    """
    @Description: split data into features and labels
    @Param: df, pandas dataframe with columns starting with X for features and Y for labels
    @Return: features and labels in pandas dataframes
    """
    xs = df.filter(regex='X') # Input : X Feature
    ys = df.filter(regex='Y') # Output : Y Feature
    return xs, ys

In [6]:
def zero_variance(df):
    """
    @Description: check for zero_variance
    @Param1: df, pandas dataframe
    @Return: names of the columns with zero variance
    """
    result = []
    for col in df.columns:
        if df[col].var() == 0:
            result.append(col)
    return result

In [7]:
def get_top_correlation(df, n=10):
    """
    @Description: print out top correlated features
    @Param1: df, pandas dataframe
    @Param2: n, number of lines to print 
    @Return: pandas series
    """
    pairs = set()
    for idx1 in range(0, df.shape[1]):
        for idx2 in range(0, idx1+1):
            pairs.add((df.columns[idx1], df.columns[idx2]))
    corr = df.corr().abs().unstack()
    corr = corr.drop(labels=pairs).sort_values(ascending=False)
    return corr[0:n]

In [8]:
def lg_nrmse(gt, preds):
    """
    @Description: Metric used in this project
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    preds = pd.DataFrame(preds)
    all_nrmse = []
    for idx in range(0,14):
        rmse = mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [9]:
train_x = pd.read_csv('data/train.csv')

test_x = pd.read_csv('data/test.csv')
train_x, train_y = dataset_split_X_y(train_x)

test_x = test_x.drop('ID', axis=1)

In [10]:
def outlier_iqr(tmp, i):
    data = tmp[i]
    
    print(np.percentile(data,75))
    print(np.percentile(data,25))
    q25, q75 = np.percentile(data, 25), np.percentile(data,75)
    iqr = q75 - q25
    
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('변수 명 : ',i)
    print('IQR : ', iqr)
    print('lower bound : ', lower)
    print('upper bound : ', upper)
    
    tmp['X_57'] = np.where(data < upper, 
                    np.where(tmp['X_57'] == 0, 0, 1), 
                    np.where(tmp['X_57'] == 0, 1, 1))
    tmp['X_57'] = np.where(data>lower, 
                    np.where(tmp['X_57'] == 0, 0, 1), 
                    np.where(tmp['X_57'] == 0, 1, 1))
    
    print("tmp['X_58'].value_counts() : ", tmp['X_57'].value_counts());
    return tmp

In [11]:
X_57 = [0 for i in range(0, train_x.shape[0])]
len(X_57)
X_57 = pd.DataFrame(X_57)
train_x['X_57'] = X_57

In [12]:
# 2차 시도 (y와 corr 기준) fixed 2 !!!!!!!!!!
cols = ["X_14", "X_17",  "X_19", "X_20", 
        "X_24", "X_25", "X_27", "X_28","X_29",  "X_41","X_42", "X_44","X_45"]


In [13]:
for i in cols:
    train_x = outlier_iqr(train_x, i)

    

X_57 = [0 for i in range(0, test_x.shape[0])]
X_57 = pd.DataFrame(X_57)
test_x['X_57'] = X_57

for i in cols:
    test_x = outlier_iqr(test_x, i)


13.39
13.35
변수 명 :  X_14
IQR :  0.040000000000000924
lower bound :  13.29
upper bound :  13.450000000000003
tmp['X_58'].value_counts() :  0    39143
1      464
Name: X_57, dtype: int64
13.53
13.5
변수 명 :  X_17
IQR :  0.02999999999999936
lower bound :  13.455000000000002
upper bound :  13.575
tmp['X_58'].value_counts() :  0    38689
1      918
Name: X_57, dtype: int64
3.31
3.16
변수 명 :  X_19
IQR :  0.1499999999999999
lower bound :  2.9350000000000005
upper bound :  3.535
tmp['X_58'].value_counts() :  0    38538
1     1069
Name: X_57, dtype: int64
3.27
3.1
변수 명 :  X_20
IQR :  0.16999999999999993
lower bound :  2.845
upper bound :  3.525
tmp['X_58'].value_counts() :  0    38525
1     1082
Name: X_57, dtype: int64
2.14
2.09
변수 명 :  X_24
IQR :  0.050000000000000266
lower bound :  2.0149999999999997
upper bound :  2.2150000000000007
tmp['X_58'].value_counts() :  0    38463
1     1144
Name: X_57, dtype: int64
2.12
2.07
변수 명 :  X_25
IQR :  0.050000000000000266
lower bound :  1.9949999999999994
u

In [21]:
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']),
        'min_child_samples': int(params['min_child_samples']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'subsample': '{:.3f}'.format(params['subsample']),
        'min_split_gain': '{:.3f}'.format(params['min_split_gain']),
        'scale_pos_weight': '{:.3f}'.format(params['scale_pos_weight']),
        'reg_alpha': '{:.3f}'.format(params['reg_alpha']),
        'reg_lambda': '{:.3f}'.format(params['reg_lambda']),
        'learning_rate': '{:.3f}'.format(params['learning_rate']),
        
    }
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1, random_state = 1, **params))
    
    loss = -cross_val_score(model, train_x, train_y, cv=10, scoring=make_scorer(lg_nrmse, greater_is_better=False)).mean()
    print("NRMSE Loss {:.5f} params {}".format(loss, params))
    return loss

In [26]:
space = {
    'n_estimators' : hp.quniform('n_estimators', 100, 1500, 50),
    'max_depth': hp.quniform('max_depth', 3, 100, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 100, 10),
    'min_child_samples': hp.quniform('min_child_samples', 10, 300, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'subsample': hp.uniform('subsample', 0.3, 1.0),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.7),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 100),
    'reg_lambda': hp.uniform('reg_lambda', 0, 100),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
}

In [15]:
space = {
    'n_estimators' : hp.quniform('n_estimators', 100, 1500, 50),
    'max_depth': hp.quniform('max_depth', 3, 100, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 100, 10),
    'min_child_samples': hp.quniform('min_child_samples', 10, 300, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'subsample': hp.uniform('subsample', 0.3, 1.0),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.7),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 100),
    'reg_lambda': hp.uniform('reg_lambda', 0, 100),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
}

best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 200,
            rstate=np.random.default_rng(1))

NRMSE Loss 1.94458 params {'n_estimators': 1350, 'max_depth': 47, 'num_leaves': 20, 'min_child_samples': 240, 'colsample_bytree': '0.715', 'subsample': '0.657', 'min_split_gain': '0.655', 'scale_pos_weight': '4.117', 'reg_alpha': '22.649', 'reg_lambda': '55.509', 'learning_rate': '0.042'}
NRMSE Loss 1.94488 params {'n_estimators': 1500, 'max_depth': 13, 'num_leaves': 90, 'min_child_samples': 110, 'colsample_bytree': '0.538', 'subsample': '0.901', 'min_split_gain': '0.652', 'scale_pos_weight': '7.202', 'reg_alpha': '1.693', 'reg_lambda': '75.762', 'learning_rate': '0.159'}
NRMSE Loss 1.95272 params {'n_estimators': 250, 'max_depth': 67, 'num_leaves': 80, 'min_child_samples': 170, 'colsample_bytree': '0.387', 'subsample': '0.902', 'min_split_gain': '0.436', 'scale_pos_weight': '8.879', 'reg_alpha': '86.379', 'reg_lambda': '88.854', 'learning_rate': '0.039'}
NRMSE Loss 1.95558 params {'n_estimators': 600, 'max_depth': 27, 'num_leaves': 60, 'min_child_samples': 120, 'colsample_bytree': '0.

NRMSE Loss 1.93862 params {'n_estimators': 800, 'max_depth': 93, 'num_leaves': 70, 'min_child_samples': 70, 'colsample_bytree': '0.820', 'subsample': '0.532', 'min_split_gain': '0.281', 'scale_pos_weight': '3.814', 'reg_alpha': '25.499', 'reg_lambda': '59.097', 'learning_rate': '0.014'}
NRMSE Loss 1.94698 params {'n_estimators': 150, 'max_depth': 80, 'num_leaves': 80, 'min_child_samples': 30, 'colsample_bytree': '0.926', 'subsample': '0.926', 'min_split_gain': '0.482', 'scale_pos_weight': '4.811', 'reg_alpha': '34.800', 'reg_lambda': '3.845', 'learning_rate': '0.181'}
NRMSE Loss 1.94046 params {'n_estimators': 350, 'max_depth': 56, 'num_leaves': 70, 'min_child_samples': 10, 'colsample_bytree': '0.593', 'subsample': '0.847', 'min_split_gain': '0.254', 'scale_pos_weight': '2.477', 'reg_alpha': '40.379', 'reg_lambda': '87.848', 'learning_rate': '0.065'}
NRMSE Loss 1.95466 params {'n_estimators': 1300, 'max_depth': 68, 'num_leaves': 80, 'min_child_samples': 170, 'colsample_bytree': '0.680'

NRMSE Loss 1.93991 params {'n_estimators': 200, 'max_depth': 22, 'num_leaves': 70, 'min_child_samples': 60, 'colsample_bytree': '0.845', 'subsample': '0.944', 'min_split_gain': '0.068', 'scale_pos_weight': '2.736', 'reg_alpha': '27.314', 'reg_lambda': '53.983', 'learning_rate': '0.024'}
NRMSE Loss 1.93258 params {'n_estimators': 1100, 'max_depth': 76, 'num_leaves': 100, 'min_child_samples': 40, 'colsample_bytree': '0.700', 'subsample': '0.869', 'min_split_gain': '0.157', 'scale_pos_weight': '5.068', 'reg_alpha': '15.868', 'reg_lambda': '24.760', 'learning_rate': '0.012'}
NRMSE Loss 1.94548 params {'n_estimators': 450, 'max_depth': 63, 'num_leaves': 20, 'min_child_samples': 20, 'colsample_bytree': '0.499', 'subsample': '0.480', 'min_split_gain': '0.114', 'scale_pos_weight': '3.394', 'reg_alpha': '77.562', 'reg_lambda': '70.889', 'learning_rate': '0.065'}
NRMSE Loss 1.93389 params {'n_estimators': 650, 'max_depth': 72, 'num_leaves': 80, 'min_child_samples': 130, 'colsample_bytree': '0.97

NRMSE Loss 1.94480 params {'n_estimators': 250, 'max_depth': 73, 'num_leaves': 90, 'min_child_samples': 90, 'colsample_bytree': '0.811', 'subsample': '0.743', 'min_split_gain': '0.195', 'scale_pos_weight': '9.183', 'reg_alpha': '61.990', 'reg_lambda': '48.522', 'learning_rate': '0.103'}
NRMSE Loss 1.94313 params {'n_estimators': 600, 'max_depth': 78, 'num_leaves': 50, 'min_child_samples': 70, 'colsample_bytree': '0.334', 'subsample': '0.956', 'min_split_gain': '0.410', 'scale_pos_weight': '6.989', 'reg_alpha': '5.040', 'reg_lambda': '65.095', 'learning_rate': '0.138'}
NRMSE Loss 1.95242 params {'n_estimators': 300, 'max_depth': 47, 'num_leaves': 80, 'min_child_samples': 90, 'colsample_bytree': '0.917', 'subsample': '0.539', 'min_split_gain': '0.262', 'scale_pos_weight': '1.094', 'reg_alpha': '72.164', 'reg_lambda': '90.731', 'learning_rate': '0.332'}
NRMSE Loss 1.93929 params {'n_estimators': 850, 'max_depth': 60, 'num_leaves': 70, 'min_child_samples': 130, 'colsample_bytree': '0.538',

NRMSE Loss 1.94363 params {'n_estimators': 100, 'max_depth': 37, 'num_leaves': 100, 'min_child_samples': 70, 'colsample_bytree': '0.855', 'subsample': '0.472', 'min_split_gain': '0.224', 'scale_pos_weight': '9.451', 'reg_alpha': '47.878', 'reg_lambda': '73.074', 'learning_rate': '0.069'}
NRMSE Loss 1.95008 params {'n_estimators': 350, 'max_depth': 24, 'num_leaves': 80, 'min_child_samples': 250, 'colsample_bytree': '0.352', 'subsample': '0.743', 'min_split_gain': '0.189', 'scale_pos_weight': '7.223', 'reg_alpha': '83.520', 'reg_lambda': '84.689', 'learning_rate': '0.048'}
NRMSE Loss 1.94252 params {'n_estimators': 450, 'max_depth': 33, 'num_leaves': 70, 'min_child_samples': 210, 'colsample_bytree': '0.952', 'subsample': '0.417', 'min_split_gain': '0.459', 'scale_pos_weight': '4.876', 'reg_alpha': '8.564', 'reg_lambda': '59.131', 'learning_rate': '0.098'}
NRMSE Loss 1.94233 params {'n_estimators': 500, 'max_depth': 48, 'num_leaves': 30, 'min_child_samples': 20, 'colsample_bytree': '0.928

NRMSE Loss 1.94989 params {'n_estimators': 1000, 'max_depth': 34, 'num_leaves': 80, 'min_child_samples': 100, 'colsample_bytree': '0.306', 'subsample': '0.734', 'min_split_gain': '0.496', 'scale_pos_weight': '6.318', 'reg_alpha': '74.601', 'reg_lambda': '38.888', 'learning_rate': '0.021'}
NRMSE Loss 1.93843 params {'n_estimators': 1300, 'max_depth': 38, 'num_leaves': 90, 'min_child_samples': 130, 'colsample_bytree': '0.423', 'subsample': '0.546', 'min_split_gain': '0.041', 'scale_pos_weight': '2.230', 'reg_alpha': '28.874', 'reg_lambda': '45.252', 'learning_rate': '0.043'}
NRMSE Loss 1.93355 params {'n_estimators': 750, 'max_depth': 6, 'num_leaves': 100, 'min_child_samples': 50, 'colsample_bytree': '0.639', 'subsample': '0.601', 'min_split_gain': '0.067', 'scale_pos_weight': '3.287', 'reg_alpha': '4.890', 'reg_lambda': '16.995', 'learning_rate': '0.030'}
NRMSE Loss 1.92933 params {'n_estimators': 600, 'max_depth': 32, 'num_leaves': 80, 'min_child_samples': 20, 'colsample_bytree': '0.59

NRMSE Loss 1.93030 params {'n_estimators': 800, 'max_depth': 13, 'num_leaves': 70, 'min_child_samples': 50, 'colsample_bytree': '0.682', 'subsample': '0.534', 'min_split_gain': '0.002', 'scale_pos_weight': '4.553', 'reg_alpha': '9.628', 'reg_lambda': '49.195', 'learning_rate': '0.018'}
NRMSE Loss 1.93576 params {'n_estimators': 550, 'max_depth': 8, 'num_leaves': 60, 'min_child_samples': 230, 'colsample_bytree': '0.752', 'subsample': '0.554', 'min_split_gain': '0.037', 'scale_pos_weight': '5.815', 'reg_alpha': '1.905', 'reg_lambda': '9.665', 'learning_rate': '0.028'}
NRMSE Loss 1.95508 params {'n_estimators': 350, 'max_depth': 3, 'num_leaves': 90, 'min_child_samples': 20, 'colsample_bytree': '0.576', 'subsample': '0.651', 'min_split_gain': '0.141', 'scale_pos_weight': '4.929', 'reg_alpha': '61.928', 'reg_lambda': '50.695', 'learning_rate': '0.021'}
NRMSE Loss 1.94705 params {'n_estimators': 300, 'max_depth': 50, 'num_leaves': 80, 'min_child_samples': 70, 'colsample_bytree': '0.805', 'su

100%|██████████████████████████████████████████| 200/200 [10:08:53<00:00, 182.67s/trial, best loss: 1.9289297361062878]


In [32]:
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_samples'] = int(best['min_child_samples'])

In [33]:
model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1, random_state = 1, **best))
model.fit(train_x, train_y)
preds = model.predict(test_x)

In [36]:
submit = pd.read_csv('data/sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
submit.to_csv('data/submission_3.csv', index = False)